In [1]:
import os
import pandas as pd
from pymongo import MongoClient # MongoDB

In [ ]:
# Connexion à MongoDB
client = MongoClient("mongodb://localhost:27017/Scribio")  # Url de connexion à la base de données MongoDB
db = client["NoSQL"]  # Nom de la base de données

In [3]:
# Récupération des chemins des fichiers CSV
chemin = "..\Scrapping\CSV"  # Chemin du dossier contenant les fichiers CSV
fichiers_csv = [os.path.join(chemin, f) for f in os.listdir(chemin) if f.endswith('.csv')]  # Liste des fichiers CSV
fichiers_csv

['..\\Scrapping\\CSV\\LivresBiographies.csv',
 '..\\Scrapping\\CSV\\LivresCorrespondances et memoires.csv',
 '..\\Scrapping\\CSV\\LivresPieces de theatre.csv',
 '..\\Scrapping\\CSV\\LivresPoesie.csv',
 '..\\Scrapping\\CSV\\LivresRecits de voyages.csv',
 '..\\Scrapping\\CSV\\LivresRoman Historique.csv']

In [4]:
# Concaténation des fichiers CSV dans un DataFrame
df_livres = pd.DataFrame()
for chemin_fichier_csv in fichiers_csv:
    df_csv = pd.read_csv(chemin_fichier_csv, sep=',', encoding='utf-8')
    df_livres = pd.concat([df_livres, df_csv])

# Nettoyage des données
df_livres = df_livres.reset_index(drop=True)
df_livres = df_livres.dropna(subset=['Nom'])
df_livres = df_livres.drop_duplicates()
print(df_livres.shape)
df_livres.head()

(431, 7)


,Nom,Prix,Description,Isbn,Photo,Editeur,Categorie
0,Mademoiselle Déjazet,14.05,"Treizième enfant d’une famille modeste, Virgin...",978-2957252107,https://m.media-amazon.com/images/I/716FfftHrG...,NaN,Biographies
1,Respire Encore: Le souffle de la résilience,10.42,NaN,979-1041562213,https://m.media-amazon.com/images/I/71G7ivVoCX...,Autoédité (23 janvier 2025),Biographies
2,Quand arrivent les chevaux,12.44,"« Mon p’tit rat, J’espère que toutes mes histo...",978-2213701516,https://m.media-amazon.com/images/I/71h73D5HAN...,Fayard (15 janvier 2025),Biographies
3,Plus rien ne pourra me blesser: Maîtrisez votr...,18.45,"Pour David Goggins, l&#39enfance a été un cauc...",978-2377530670,https://m.media-amazon.com/images/I/81wEK7MtAy...,NIMROD (28 septembre 2023),Biographies
4,Le Journal d&#39Anne Frank (Nouvelle édition),15.12,NaN,978-2253937432,https://m.media-amazon.com/images/I/71DKiNxs0d...,Le Livre de Poche; 1er édition (25 mai 2022),Biographies


In [5]:
tab_livres_json = []
for i, row in df_livres.iterrows():
    livre = {
        "Nom": row["Nom"],
        "Prix": row["Prix"],
        "Description": row["Description"],
        "Isbn": row["Isbn"],
        "Photo": row["Photo"],
        "Editeur": row["Editeur"],
        "Categorie": row["Categorie"]
    }
    tab_livres_json.append(livre)
tab_livres_json

[{'Nom': 'Mademoiselle Déjazet',
  'Prix': 14.05,
  'Description': 'Treizième enfant d’une famille modeste, Virginie Déjazet débute sur les planches à l’âge de cinq ans. Le succès semble rapidement lui sourire, mais la presse et les rivalités sont redoutables sur le célèbre Boulevard du Crime…\nFemme de tête et femme de cœur, elle se forge une devise "Bien faire et laisser dire". Mais la course à l’amour fou et au succès n’est pas sans écueils. Comment rester soi-même dans un milieu où la concurrence est rude et se joue dans le lit des directeurs de théâtre ?\nLe rideau n’attend plus que vous pour se lever sur l’histoire vraie de Virginie Déjazet, la petite comédienne qui côtoya Rachel, Sarah Bernhardt et toute l’élite intellectuelle de son temps, avant d’être balayée par les soubresauts d’un siècle tourmenté.',
  'Isbn': '978-2957252107',
  'Photo': 'https://m.media-amazon.com/images/I/716FfftHrGL._SY522_.jpg',
  'Editeur': nan,
  'Categorie': 'Biographies'},
 {'Nom': 'Respire Encore:

In [6]:
# Insertion des livres dans la collection Livres de la base de données NoSQL
db["Livres"].insert_many(tab_livres_json)